# Neural Network Trained with Genetic Algorithm Made in Java/Processing for the Game Snake

#### Camilo Gómez Zapata - C.C. 1010088965 - Neural Networks Course

### Introduction

This project consists in the creation of an intelligent agent capable of playing the game Snake. The agent is created with a neural network architecture and uses the principles of deep learning and genetic algorithms to train itself in the game through continous iterations. It is developed in a Java library called Processing, which is made for representing graphics on screen.

### Setting up the project

Before explaining how everything was developed, these are the steps required to run it locally:
- Download Processing from this link: https://processing.org/download
- Extract anywhere. There's no need for installation.
- Add Processing to the PATH, which is a simple process following these instructions: https://github.com/TobiahZ/processing-vscode#add-processing-to-path
- Find the folder `GeneticSnake` which is in the same route than this file, and execute this command:

`processing-java --force '--sketch=<complete-route-to-GeneticSnake>/GeneticSnake' '--output=<complete-route-to-GeneticSnake>/GeneticSnake/out' --run`

Remember to replace `<complete-route-to-GeneticSnake>` for the complete route to the file. In my case I replaced it for `C:/Users/Camilo/Documents/NeuralNetworkRSouzaCourse/Notebooks/extras/HomeworksSubmissions/1010088965/Final_Project`.

That should execute everything, and it should look like this:

<center><img src="img/setup.png" width=800></src></center>

The user can interact with the game by increasing or decreasing the speed of the iterations, changing the mutation rate, and rendering all snakes that are being trained simultaneously. This can be done by pressing the keys that appear on screen. Each of these features will be explained in the next sections.

The agent starts showing progress usually by the 70th generation, although in some cases it might take longer.

### Creating the game

Before creating the AI it is necessary to create the game. The game itself is a standard Snake game where the snake can move in four different directions (vertically and horizontally), and getting a piece of food increases its score by one. The game grid is of size 36x27. The objective of the game, as usual, is to increase the size of the snake as large as possible so it can grow and get to fill the entire screen. The main file, through which all of the logic is run, is the `GeneticSnake.pde` file, because it has the same name of the folder. In this file you can see a `setup()` function, which is the one that initializes everything, and then a `draw()` function, which is the game loop that runs each new frame.

The complete logic for just the Snake game can be found in the files `World.pde`, `Snake.pde` and `Food.pde`. Of course, all of these files also contain a bit of the logic for the AI training, but they're mostly the logic for the basic game. In the `Snake.pde` file, which is a class that creates a single `Snake` object, you can see that there is the function `update()` which is the one that iterates the snake in every new frame. This function executes everything for the snake, like updating its position and the position of its body, checking if the snake ate food to increase its size and score, and checking if it collided with itself or with the world to know it died. The `render()` function in there is the one that draws the snake on the screen in each new frame.

The most complex part of implementing the game was implementing the logic to move the body, because given that the snake is always moving, in each new frame a square of the body must be redrawn in the place the next square closest to the head was in the previous frame, and this must be done with all of the squares. This is implemented in the `move()` function, and requires a previous copy of the snake to be kept in all frames.

### Neural Network

After implementing the basic game, the next step is giving the snake a brain. This brain is a neural network which will control all of its movements. The training process of the brain will be explained in the next section. Here the explanation for this neural network is given. All of the logic necessary for the brain is implemented in the `NeuralNet.pde`, `Matrix.pde` and `Controller.pde` classes.

As you can see in the `Snake.pde` file, the class always initializes a `brain` property, which is an instance of the class `NeuralNet`. This means that every snake always has its corresponding unique brain. Here, the NeuralNet class code was implemented from scratch, and it is implemented so it always has four layers: one input layer, two hidden layers, and one output layer. As you can see in the `NeuralNet.pde` file, everything is implemented from basic principles. In the constructor of the class the weights and biases are initialized as matrices with random values, and that's where the `Matrix.pde` file comes in, which holds all of the logic for matrix operations.

Each time an instance of this class is initalized in the `Snake` class it receives the arguments `(24, 24, 4)` which are the numbers of neurons in each layer. This means that each snake receives 24 inputs from the world, and outputs the 4 different directions in which it can move. This happens in each frame.

<center><img src="img/dirs.png" width=640></src></center>

The picture shows that the Snake can look in 8 different directions, and in each one of these directions it looks for three pieces of data: distance to the wall, distance to the food (if there is any), and distance to itself. This logic of looking in each direction can be found in the `Controller.pde` file. This class is the one that uses the snake's own brain to control its movements. The class is supposed to be analogous to what a human player would do playing the game, but using the snake's own brain. In the class you can find the functions `play()` which triggers the logic for the snake's brain in each frame. In this function, the eyesight of the snake is obtained from the function `frame()` inside of the same class, which in turn runs the function `lookInDirection()`, which gets the three values in each direction that was looked at by the snake. These three values are classified like this:

- Looking at the food: either $1$ if there's food or $0$ if there isn't.
- Looking at the walls: $1/\mathrm{distanceToWall}$.
- Looking at its own body: $1/\mathrm{distanceToBody}$ if it finds the body, or $0$ if it doesn't.

After the snake receives 24 different values of its eyesight in the `frame()` function, it passes it to the brain through its `output()` function. All of this is done in the `Controller`'s `play()` function. The `output()` function of the brain is defined in the `NeuralNet` class, and is esentially the function in charge of running the feed-forward algorithm.

If you look into this `output()` function, you can see all of the feed-forward steps, which basically consist in all of the matrix multiplications between the layers, and the addition of the biases for each layer:


    // First layer
    Matrix inputs = woh.singleColumnMatrixFromArray(inputsArr); // convert input array to matrix (independent of woh)
    Matrix inputsBias = inputs.addBias(); // add bias of inputs

    // Second layer (first hidden one)
    Matrix hiddenInputs = whi.dot(inputsBias);
    Matrix hiddenOutputs = hiddenInputs.activate();
    Matrix hiddenOutputsBias = hiddenOutputs.addBias();

    // Third layer (second hiddden one)
    Matrix hiddenInputs2 = whh.dot(hiddenOutputsBias);
    Matrix hiddenOutputs2 = hiddenInputs2.activate();
    Matrix hiddenOutputsBias2 = hiddenOutputs2.addBias();

    // Last layer, which is output
    Matrix outputInputs = woh.dot(hiddenOutputsBias2);
    Matrix outputs = outputInputs.activate();


The first layer is just the inputs with the biases of the nodes. The second layer is calculated as the matrix multiplication between the matrix with the weights between the first and second layer, and the matrix with the input nodes with their biases. Then an activation function is applied in `hiddenInputs.activate()`. In the `Matrix.pde` it can be found that this activation function is sigmoid. The third layer is the same thing, but with the previous layer of nodes, and the output layer is also the same thing, but that one doesn't have biases.

This function generates the output of the neural net, and this is done in each frame. Finally, again in the `Controller` class you can see in the `play()` function that this output is checked to see which of the four variables got the highest number, and the index with this highest number represents the direction in which the snake is going to move in that frame.

### Genetic Algorithm

As you may have noticed, there is no implementation of backpropagation in the `NeuralNet` class. This is because backpropagation is not the method used to train the network. The method used to train it is a genetic algorithm, and in this section the way it works will be explained.

A genetic algorithm is a learning technique that was designed to resemble the evolution of real species in nature. To do this, the program at the beginning initializes a population of individuals with randomly generated brains. Then, it lets each individual interact with the world until it dies. At the moment of death, a score (fitness) is assigned to each individual, which measures its performance during the game. At the end, when all individuals are dead, a new generation is created by combining the brains of the individuals of the previous generation, giving priority to the ones that got the highest score so the best ones get to spread their genes. This process is repeated through many generations, as many as are required, until the training starts generating good individuals capable of getting high scores.

In the code, all of this is implemented in the `Population.pde`, `Generation.pde` and `Foods.pde` classes. At the beginning of the game, you can see in the `GeneticSnake.pde` file that a new `Population` is created within the `setup()` function. This `Population` is going to contain all of the generations. You can see in the constructor of the `Population` class that it initializes a new instance of `Generation`, and then you can see in the constructor of the `Generation` class that it generates a certain amount of snakes, which in this case is set to 2000 per generation.

This means that when the game starts, always a new population is created with its first generation of 2000 snakes, with all of them having 2000 different brains. Then, you can see in the `draw()` function that the line `population.update()` is run in each frame, and this `update()` function of the `Population` class is essentially in charge of playing the 2000 snakes. You can see that this function is composed of a `for` loop which iterates over all snakes and runs the individual `update()` function of each one.

This means that all snakes are trained simultaneously, and you can see this in the simulation! If you run the game and press the letter R you can see all of the snakes training at the same time:

<center><img src="img/all-snakes.png" width=640></src></center>

In the picture you can see all of the snakes at the beginning, which are all initialized with random directions, moving and training themselves.

One small detail added to the simulation just for showcase purposes was that at the end of each generation, the snake that got the highest score in the generation is brought back to life just to demonstrate it. This is done through the `playBestSnake()` function inside the `update()` function of the `Population` class.

After the best snake dies for the second time, you can see inside of the `playBestSnake()` function that the `changeGen()` function is executed. This function is in charge of making the generation switch, and combining the brains of the last one to create the new one. `changeGen()` is an important function, esentially in charge of the genetic algorithm. These are its steps:

1. First it needs to obtain the fitness of each snake. The fitness function we define is very important because it's the one that contains the information necessary to know which snakes to prioritize and which ones not. In this case, you can see in the `died()` function of the `Snake` class that it is calculated like this:

    If the length of the snake is lesser than $10$:

    $$\mathrm{fitness}=\mathrm{lifetime}\times 2^{\mathrm{length}}$$

    Else:

    $$\mathrm{fitness}=\mathrm{lifetime}\times 2^{10}\times (\mathrm{length}-9)$$

    This makes it so the fitness function gives priority to time alive, but more importantly, when the snake is initially growing it gives exponential priority to the size of the snake. If it's capable of increasing size it should have a very higher probability of spreading its genes. But when the snake is past size 10, then that exponential priority would grow too large so it just continues growing linearly.

2. After obtaining the fitness of each snake, sum all of the fitnesses to have a total fitness of the generation.

3. Create a new generation of snakes with all of their brains initilized randomly.

4. Iterate over all snakes in the new generation. For each snake:

    1. Select one snake from the last generation as first parent. The probability of picking a snake is calculated as $\frac{\mathrm{snakeFitness}}{\mathrm{totalFitness}}$

    2. Select another snake from the last generation as second parent, using the same probability.

    3. Crossover the brain of the first parent with the brain of the second one. The brains are just instances of the class `NeuralNet`, and you can see in the file that there is a `crossover()` function, which is composed of the crossovers of all of the weights and biases matrices. Then you can see in the `crossover()` function of the `Matrix` class that all it does is to create a new matrix from the elements of the two parent ones, picking randomly which elements to obtain from one matrix and which elements from the other one.

    4. Make the brain of the snake to be the result of the crossover of the previous step. This deletes the previously created brain with random values.

    5. Mutate the brain. This is included so the process can always have random mutations which might result in better performances. You can see that there is a `mutate()` function in the `NeuralNet` class, which mutates each of the weights and biases matrices, which also have their own `mutate()` function, which receives as a parameter mutationRate `mr` that represents the percentage of the values in the matrix that will be switch for a randomly picked value. This mutation factor can be changed by the user in the middle of the simulation, with the S and D keys.

5. Delete the old generation to free up memory (optional).

Following this process, the new generation is left ready for execution. Then the `update()` function of `Population` plays each snake the same way than before until all of them are dead, and a new generation is created the same way, and so on the cycle continues.

Usually, around 60 to 80 generations are needed for the snakes to start to show satisfactory results.

<center><video width=800 controls src="img/vid.mp4"></video></center>

This process can be left to be continued for as long as the user wants, and it effectively is capable of training the networks.

<center><img src="img/git.png" width=640></src></center>

In the picture there is an example of a trained snake in the 73rd generation.

### Conclusions and next steps

It has been shown here that the genetic algorithm is an effective way to train neural networks, proving by example with the application of intelligent agents that are capable of playing the game of Snake. After many iterations, all simulations usually find the agents capable of reaching high scores, although it is interesting to notice that there are usually many generations at the start of each simulation where the snakes don't achieve much. From this it can be concluded that the capability of the agents to have random mutations at the switch of each generation is important, because otherwise the brains would be stuck with the same initial parameters of the first generation, which are probably not ideal.

An interesting next step would be to try other forms of training of the neural nets and check which ones of them perform the best. Also, it would be interesting to change the output of the net, so instead of having four possible directions to move it should have only two, because the only thing that really matters is turning left or right relative to the snake's direction. Another interesting improvement could be to have the snakes fight each other and pick the winners of the fight to see if they perform better the task of looking for the food.

### References

- Introduction to Genetic Algorithms: https://towardsdatascience.com/introduction-to-genetic-algorithms-including-example-code-e396e98d8bf3
- Introduction to Neural Networks: https://www.section.io/engineering-education/introduction-to-neural-networks
- Under The Hood of Neural Network Forward Propagation — The Dreaded Matrix Multiplication: https://towardsdatascience.com/under-the-hood-of-neural-network-forward-propagation-the-dreaded-matrix-multiplication-a5360b33426
- AI learns to play snake using Genetic Algorithm and Deep learning: https://www.youtube.com/watch?v=3bhP7zulFfY